### Measuring distance between positions using multiple transformations

Here we measure the distance between corners representations using different transformation on the neural data. Complete data, PC, CC and LDA transformations. 

We run a shufflinf analysis over the balanced data. The shuffling is done over different events, where we define as an event the complete visit to the corner. 

Visits are only taken if they are longer than 2s, and the balancing is done from shorter to longer events, meaning that, if necessary, we leave aside long visits to the corner.

In [1]:
#import useful libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import pickle
import configuration
import general_statistics as stats
import figures as figs
import matplotlib.cm as cm
from matplotlib import colors
from scipy import signal
from scipy import stats as sstats
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import scipy
cmap = cm.jet
import math
from sklearn.datasets import load_iris
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.cross_decomposition import CCA
from sklearn.model_selection import cross_val_score
import random
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
import seaborn as sns
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA



import warnings
warnings.filterwarnings("ignore")

In [2]:
def speed_selection(tracking = None, speed_limit = 3):
    
    x = tracking[0,:]
    y = tracking[1,:]
    vx = np.diff(x)
    vy = np.diff(y)
    speed = np.sqrt(vx*vx+vy*vy)
    index = np.where(speed > speed_limit)[0]
    return index

In [4]:
# define mouse information, and preprocessin versions from analysis.
# load data for all sessions in this mouse and compute mean activity for each neuron.

mouse = 56165      ### mouse number id
decoding_v = 1            ## decoding version, normaly equal to one
motion_correction_v = 20  ### 100 means everything was aligned, 200 means it was also equalized, 20 is for day wise
alignment_v = 3           ## alignment version, version 3 is for day wise
equalization_v = 0        ## equalization version
source_extraction_v = 1   ## source extraction version
component_evaluation_v = 1 ## component evaluation version

# here we will do analysis with no registration
# registration_v = 2        ## registration version
sf = 10                   ## sampling frequency of the original signal 
re_sf= 1                 ## value of resampling
session_now = 1
period = int(10 * sf / re_sf)
period_resting = period *50
fixed = 'None'
N_SHUFFLINGS = 50
## define task for plotting. This will cahnge for other mice!!!!
colorcodes = ['Greys','PuBu', 'YlOrRd','Reds','Blues','Oranges','Greens']
if mouse == 411857:
    sessions = [1,2,3]
    if session_now == 1:
        task = 'STABLE'
        colapse_behaviour = 2
        labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
        colornames=['k',"r","deepskyblue","g","blue","g","blue"]
    if session_now == 2:
        task = 'RANDOM'
        colapse_behaviour = 0
        labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
        colornames=['k',"r","deepskyblue","g","blue","g","blue"]   
    if session_now == 3:
        task = 'OVERLAPPING'
        fixed = 'LL'
        object_fixed = 3
        colapse_behaviour = 1
        labels =['Unlabel','Rest1', 'Navigation', 'Overlap_object' , 'Moving_object','RunOO' , 'RunMO' ]
        colornames=['k',"r","deepskyblue","g","blue","g","blue"]   


if mouse == 56165 or mouse == 32364:
    if mouse == 56165:
        sessions = [1,2,4]       ## sessions for this particular mouse
    if mouse == 32364:
        sessions = [1,2]
    if session_now == 1:
        task = 'OVERLAPPING'
        colapse_behaviour = 1
        labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
        colornames=['k',"r","deepskyblue","g","blue","g","blue"]
        if mouse == 32364:
            fixed = 'LR'
            object_fixed = 4
            colapse_behaviour = 1
            labels =['Unlabel','Rest1', 'Navigation', 'Overlap_object' , 'Moving_object','RunOO' , 'RunMO' ]
            colornames=['k',"r","deepskyblue","g","blue","g","blue"]   
        if mouse == 56165:
            fixed = 'UR'
            object_fixed = 5
            colapse_behaviour = 1
            labels =['Unlabel','Rest1', 'Navigation', 'Overlap_object' , 'Moving_object','RunOO' , 'RunMO' ]
            colornames=['k',"r","deepskyblue","g","blue","g","blue"]   

    else:
        if session_now == 2:
            task = 'STABLE'
            colapse_behaviour = 2
            labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
            colornames=['k',"r","deepskyblue","g","blue","g","blue"]
        else:
            task = 'RANDOM'
            colapse_behaviour = 0
            labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
            colornames=['k',"r","deepskyblue","g","blue","g","blue"]
            
if mouse == 32365:
    sessions = [2,3] ## sessions for this particular mouse
    if session_now == 2:
        task = 'RANDOM'
        colapse_behaviour = 0
        labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
        colornames=['k',"r","deepskyblue","g","blue","g","blue"]
    else:
        if session_now == 3:
            task = 'OVERLAPPING'
            fixed = 'LR'
            object_fixed = 4
            colapse_behaviour = 1
            labels =['Unlabel','Rest1', 'Navigation', 'Overlap_object' , 'Moving_object','RunOO' , 'RunMO' ]
            colornames=['k',"r","deepskyblue","g","blue","g","blue"]           
            
if mouse == 56166:
    sessions = [1,2] ## sessions for this particular mouse
    if session_now == 1:
        task = 'RANDOM'
        colapse_behaviour = 0
        labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
        colornames=['k',"r","deepskyblue","g","blue","g","blue"]

    else:
        if session_now == 2:
            task = 'OVERLAPPING'
            fixed = 'UR'
            object_fixed = 5
            colapse_behaviour = 1
            labels =['Unlabel','Rest1', 'Navigation', 'Overlap_object' , 'Moving_object','RunOO' , 'RunMO' ]
            colornames=['k',"r","deepskyblue","g","blue","g","blue"]
            
if mouse == 32366:
    sessions = [2,3] ## sessions for this particular mouse
    if session_now == 3:
        task = 'RANDOM'
        colapse_behaviour = 0
        labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
        colornames=['k',"r","deepskyblue","g","blue","g","blue"]
            
if mouse == 32363:
    sessions = [1,2] ## sessions for this particular mouse
    if session_now == 1:
        task = 'RANDOM'
        colapse_behaviour = 0
        labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
        colornames=['k',"r","deepskyblue","g","blue","g","blue"]
    else:
        if session_now == 2:
            task = 'OVERLAPPING'
            fixed = 'UL'
            object_fixed = 6
            colapse_behaviour = 1
            labels =['Unlabel','Rest1', 'Navigation', 'Overlap_object' , 'Moving_object','RunOO' , 'RunMO' ]
            colornames=['k',"r","deepskyblue","g","blue","g","blue"]
        

In [5]:
file_directory = os.environ['PROJECT_DIR'] + 'neural_analysis/data/calcium_activity_day_wise/'
timeline_file_dir = os.environ['PROJECT_DIR'] + 'neural_analysis/data/timeline/'
behaviour_dir = os.environ['PROJECT_DIR'] + 'calcium_imaging_behaviour/data/ethogram/' + f'{mouse}' + '/session_' + f'{session_now}' + '/'
behaviour_dir_parameters = os.environ['PROJECT_DIR'] + 'calcium_imaging_behaviour/data/ethogram_parameters/' + f'{mouse}' + '/session_' + f'{session_now}' + '/'
tracking_dir = os.environ['PROJECT_DIR'] + 'calcium_imaging_behaviour/data/center_of_mass/' + f'{mouse}' + '/session_' + f'{session_now}' + '/'
objects_dir= os.environ['PROJECT_DIR'] + 'calcium_imaging_behaviour/data/object_positions/'
figure_path = os.environ['PROJECT_DIR'] +'neural_analysis/figures/trial_aligned_events/'


In [20]:
# define all relevant files names and load them into a list
session = session_now

activity_list = []
timeline_list = []
behaviour_list = []
corners_list = []
corners_list2 = []

speed_list = []

behaviour_list_unsup = []
parameters_list = []
parameters_list2 = []
parameters_time = []
tracking_list = []
total_time = 0
day = 0

print('LOADING TRIALS ACTIVITY AND CREATING LIST OF ACTIVITY, TRACKING AND BEHAVIOUR')
for trial in [1,6,11,16]:

    beh_file_name_1 = 'mouse_' + f'{mouse}' + '_session_' + f'{session}' + '_day_' + f'{day+1}' + '_likelihood_0.75_ethogram.npy'
    ## LOAD BEHAVIOUR
    behaviour = np.load(behaviour_dir + beh_file_name_1)
    reshape_behaviour = np.reshape(behaviour[:int(int(behaviour.shape[0]/re_sf)*re_sf)],(int(behaviour.shape[0]/re_sf),re_sf))
    resample_beh1 = np.reshape(scipy.stats.mode(reshape_behaviour,axis=1)[0],reshape_behaviour.shape[0])

    beh_file_name_1 = 'mouse_' + f'{mouse}' + '_session_' + f'{session}' + '_day_' + f'{day+1}' + '_likelihood_0.75_object_corners.npy'
    ## LOAD CORNERS EXPLORATION
    behaviour = np.load(behaviour_dir + beh_file_name_1)
    reshape_behaviour = np.reshape(behaviour[:int(int(behaviour.shape[0]/re_sf)*re_sf)],(int(behaviour.shape[0]/re_sf),re_sf))
    corners = np.reshape(scipy.stats.mode(reshape_behaviour,axis=1)[0],reshape_behaviour.shape[0])

    speed_file_name = 'mouse_' + f'{mouse}' + '_session_' + f'{session}' + '_day_' + f'{day+1}' + '_likelihood_0.75_speed.npy'
    ## LOAD INSTANTANEOUS SPEED
    speed = np.load(behaviour_dir + speed_file_name)
    reshape_speed = np.reshape(speed[:int(int(behaviour.shape[0]/re_sf)*re_sf)],(int(behaviour.shape[0]/re_sf),re_sf))
    resample_speed = np.reshape(scipy.stats.mode(reshape_speed,axis=1)[0],reshape_speed.shape[0])
    
    
    beh_file_name_1 = 'mouse_' + f'{mouse}' + '_session_' + f'{session}' + '_trial_' + f'{day+1}' + '_likelihood_0.75_ethogram_parameters.npy'
    ## LOAD PARAMETRS FOR BEHAVIOUR CLASSIFICATION
    parameters = np.load(behaviour_dir_parameters + beh_file_name_1)
    
    params0 = []
    params = []
    for param in range(0,2): ## take only ALLOCENTRIC REPRESENTATION
        r1_params = np.reshape(parameters[param,:int(int(behaviour.shape[0]/re_sf)*re_sf)],(int(behaviour.shape[0]/re_sf),re_sf))
        r2_params = np.reshape(scipy.stats.mode(r1_params,axis=1)[0],reshape_behaviour.shape[0])
        r_params = parameters[param,:resample_speed.shape[0]]
        r_params[:r2_params.shape[0]] = r2_params
        params.append(r_params)
    resample_params0 = np.array(params)
    
    params = []
    for param in range(2,7): ## take only ALLOCENTRIC REPRESENTATION
        r1_params = np.reshape(parameters[param,:int(int(behaviour.shape[0]/re_sf)*re_sf)],(int(behaviour.shape[0]/re_sf),re_sf))
        r2_params = np.reshape(scipy.stats.mode(r1_params,axis=1)[0],reshape_behaviour.shape[0])
        r_params = parameters[param,:resample_speed.shape[0]]
        r_params[:r2_params.shape[0]] = r2_params
        params.append(r_params)
    resample_params = np.array(params)
    
    params2 = []
    for param in range(7,11): ## take only ALLOCENTRIC REPRESENTATION
        r1_params = np.reshape(parameters[param,:int(int(behaviour.shape[0]/re_sf)*re_sf)],(int(behaviour.shape[0]/re_sf),re_sf))
        r2_params = np.reshape(scipy.stats.mode(r1_params,axis=1)[0],reshape_behaviour.shape[0])
        r_params = parameters[param,:resample_speed.shape[0]]
        r_params[:r2_params.shape[0]] = r2_params
        params.append(r_params)
    resample_params2 = np.array(params)
    
    ## LOAD TRACKING
    tracking_file_name_1 = 'mouse_' + f'{mouse}' + '_session_' + f'{session}' + '_day_' + f'{day+1}' + '_likelihood_0.75.npy'
    position = np.load(tracking_dir + tracking_file_name_1)
    resample_position, resample_position_stc = stats.resample_matrix(neural_activity=position.T,
                                                                                            re_sf=re_sf)
    ## LOAD TIMELINE
    time_file_session_1 =  'mouse_'+ f'{mouse}'+'_session_'+ f'{session}' +'_trial_'+ f'{trial}'+'_v1.3.1.0_10.pkl'

    timeline_file= open(timeline_file_dir + time_file_session_1,'rb')
    timeline_info = pickle.load(timeline_file)
    timeline_1 = np.zeros(len(timeline_info) + 1)
    for i in range(len(timeline_info)):
        timeline_1[i] = timeline_info[i][1]
    timeline_1[len(timeline_info)] = behaviour.shape[0]
    timeline = timeline_1/re_sf
    time_lenght = 10
    resample_timeline = timeline_1/re_sf
    timeline_list.append(resample_timeline)

    behaviour_list.append(resample_beh1)
    corners_list.append(corners)
    speed_list.append(resample_speed)

    parameters_list.append(resample_params)
    parameters_list2.append(resample_params2)
    parameters_time.append(resample_params0)
    tracking_list.append(resample_position)
    total_time = total_time + behaviour.shape[0]


    file_name_session_1 = 'mouse_' + f'{mouse}' + '_session_' + f'{session}' + '_trial_'+ f'{trial}'+'_v' + f'{decoding_v}' + '.4.' + f'{motion_correction_v}' + \
                            '.' + f'{alignment_v}' + '.' + f'{equalization_v}' + '.' + f'{source_extraction_v}' + '.' + \
                          f'{component_evaluation_v}' +  '.0.npy'

     ##load activity and timeline
    activity = np.load(file_directory + file_name_session_1)
    neural_activity1 = activity[1:,:]
    ## z-score neural activity
    neural_activity = neural_activity1[:,:int(int(behaviour.shape[0]/re_sf)*re_sf)]
    ##downsample neural activity
    resample_neural_activity_mean, resample_neural_activity_std = stats.resample_matrix(neural_activity=neural_activity,
                                                                                            re_sf=re_sf)

    activity_list.append(resample_neural_activity_mean)
    print(resample_neural_activity_mean.shape)

    day = day + 1

LOADING TRIALS ACTIVITY AND CREATING LIST OF ACTIVITY, TRACKING AND BEHAVIOUR
(195, 32368)
(448, 30592)
(322, 31977)
(397, 30905)


In [22]:
day = 0
pca = PCA()
activity_list_pca = []
variance_list = []
variance_ratio_list = []

activity_list2 = []

cca_components = min(5,activity_list[day].shape[0])
cca = CCA(n_components=cca_components)
cca_components2 = 4
cca2 = CCA(n_components=cca_components2)
cca_components0 = 2
cca0 = CCA(n_components=cca_components0)

activity_list_cca = []
activity_list_cca2 = []
activity_list_cca0 = []

clf = LDA()
activity_list_lda = []

#embedding = MDS(n_components=3)

for day in range(len(behaviour_list))
    
    ### run pca on the entire dataset
    pca.fit(activity_list[day].T)
    transformed_activity = pca.fit(activity_list[day].T).transform(activity_list[day].T)
    #X_pc_transformed = embedding.fit_transform(transformed_activity.T)
    activity_list_pca.append(transformed_activity.T)
    variance_list.append(pca.explained_variance_/(1+np.sqrt(activity_list[day].shape[0]/activity_list[day].shape[1]))**2)
    normalized_variance = pca.explained_variance_/(1+np.sqrt(activity_list[day].shape[0]/activity_list[day].shape[1]))**2
    variance_ratio_list.append(np.cumsum(normalized_variance/sum(normalized_variance)))
    
    X = activity_list[day].T
    y = behaviour_list[day]
    X_transformed = clf.fit(X, y).transform(X)
    #X_lda_transformed = embedding.fit_transform(X_transformed.T)
    activity_list_lda.append(X_transformed.T)

    cca_transformed = cca0.fit(activity_list[day].T, parameters_list[day].T).transform(activity_list[day].T)
    #X_cc_transformed = embedding.fit_transform(cca_transformed.T)    
    activity_list_cca0.append(cca_transformed)
    
    cca_transformed = cca.fit(activity_list[day].T, parameters_list[day].T).transform(activity_list[day].T)
    #X_cc_transformed = embedding.fit_transform(cca_transformed.T)    
    activity_list_cca.append(cca_transformed)
    
        
    cca_transformed = cca2.fit(activity_list[day].T, parameters_list2[day].T).transform(activity_list[day].T)
    #X_cc_transformed = embedding.fit_transform(cca_transformed.T)    
    activity_list_cca2.append(cca_transformed)
    

In [23]:
    ## define targets of behaviour according to protocol (random, overlapping and stable)

    id_target = [0,1,2,3,4] # 0: unlabeled, 1:resting, 2:Navigation, 3: Obj1, 4:Obj2, 5:Run1, 6:Run2
    
    print('REDIFINING BEHAVIOUR FOR DIFFERENT SESSIONS')

    if colapse_behaviour == 0 : # RANDOM
        for day in range(len(behaviour_list)):
            for trial in range(5):
                behaviour_trial = behaviour_list[day][int(timeline_list[day][2*trial]):int(timeline_list[day][2*trial+1])]
                objects = np.unique(behaviour_trial)
                selected_object = np.random.randint(len(objects)-4,len(objects)-2,1)
                index0 = np.where(behaviour_trial==objects[selected_object])[0]
                index1 = np.where(np.logical_and(behaviour_trial==objects[len(objects)-4], behaviour_trial!=objects[selected_object]))[0]
                index2 = np.where(np.logical_and(behaviour_trial==objects[len(objects)-3], behaviour_trial!=objects[selected_object]))[0]
                behaviour_trial[index0] = 3
                behaviour_trial[index1] = 4

                behaviour_trial[index2] = 4            

                index0 = np.where(behaviour_trial==objects[selected_object]+4)[0]
                index1 = np.where(np.logical_and(behaviour_trial==objects[len(objects)-2], behaviour_trial!=objects[selected_object]+4))[0]
                index2 = np.where(np.logical_and(behaviour_trial==objects[len(objects)-1], behaviour_trial!=objects[selected_object]+4))[0]
                behaviour_trial[index0] = 0
                behaviour_trial[index1] = 0
                behaviour_trial[index2] = 0 

                behaviour_list[day][int(timeline_list[day][2*trial]):int(timeline_list[day][2*trial+1])] = behaviour_trial


    if colapse_behaviour == 1 : #OVERLAPPING
        for day in range(len(behaviour_list)):
            behaviour_list[day][np.where(behaviour_list[day] == object_fixed)[0]] = 100
            behaviour_list[day][np.where(np.logical_and(behaviour_list[day]>=3, behaviour_list[day]<=6))[0]] = 4
            behaviour_list[day][np.where(behaviour_list[day] == 100)[0]] = 3        
            behaviour_list[day][np.where(behaviour_list[day] == object_fixed +4)[0]] = 0        
            behaviour_list[day][np.where(np.logical_and(behaviour_list[day]>=7, behaviour_list[day]<=10))[0]] = 0
            behaviour_list[day][np.where(behaviour_list[day] == 200)[0]] = 0


    if colapse_behaviour == 2: #STABLE
        for day in range(len(behaviour_list)):
            objects = np.unique(behaviour_list[day])
            selected_object = np.random.randint(len(objects)-4,len(objects)-2,1)
            index0 = np.where(behaviour_list[day]==objects[selected_object])[0]
            index1 = np.where(np.logical_and(behaviour_list[day]==objects[len(objects)-4], behaviour_list[day]!=objects[selected_object]))
            index2 = np.where(np.logical_and(behaviour_list[day]==objects[len(objects)-3], behaviour_list[day]!=objects[selected_object]))
            behaviour_list[day][index0] = 3
            behaviour_list[day][index1] = 4
            behaviour_list[day][index2] = 4      

            index0 = np.where(behaviour_list[day]==objects[selected_object]+4)[0]
            index1 = np.where(np.logical_and(behaviour_list[day]==objects[len(objects)-2], behaviour_list[day]!=objects[selected_object]+4))
            index2 = np.where(np.logical_and(behaviour_list[day]==objects[len(objects)-1], behaviour_list[day]!=objects[selected_object]+4))
            behaviour_list[day][index0] = 0
            behaviour_list[day][index1] = 0
            behaviour_list[day][index2] = 0  

REDIFINING BEHAVIOUR FOR DIFFERENT SESSIONS


In [24]:
print('CREATING NAVIGATION OF CORNERS AND EXPLORATION OF CORNERS VECTORS')

navigation_list = [] 
exploration_list = []

SPEED_LIMIT = 5
for day in range(len(behaviour_list)):
    speed = signal.medfilt(speed_list[day],9)
    # select the corners that are being explored when the animal is doing an exploratory task
    explorating_object = np.zeros_like(corners_list[day])
    explorating_object[np.where(behaviour_list[day]==4)[0]] = corners_list[day][np.where(behaviour_list[day]==4)[0]]
    explorating_object[np.where(behaviour_list[day]==3)[0]] = corners_list[day][np.where(behaviour_list[day]==3)[0]]
    #exploration_corner = np.zeros_like(explorating_object)
    # create a vector that contains zeros everywhere but object ID number when the animal is exploring an object a particular corner with low speed
    #exploration_corner[np.where(speed<SPEED_LIMIT)[0]] = navigation[np.where(speed<SPEED_LIMIT)[0]]
    #exploration_corner = explorating_object
    
    navigation_corner = np.zeros_like(corners_list[day])
    for corner in [1,2,3,4]:
        # create a vector that contains zeros everywhere but corner ID when the animal is navigation at that position with out an object
        navigations_at_corner = np.logical_and(explorating_object==0,corners_list[day]==corner)
        #print(len(np.where(navigations_at_corner)[0]))
        #navigation_corner[np.logical_and(navigations_at_corner, speed < SPEED_LIMIT)] = corner
        navigation_corner[navigations_at_corner] = corner
        
    navigation_list.append(navigation_corner)
    exploration_list.append(explorating_object)

CREATING NAVIGATION OF CORNERS AND EXPLORATION OF CORNERS VECTORS


In [25]:
print('CREATES LIST THAT SAVES ALL THE EVENTS IN A DAY AND CONTAINS ONSET OF VISITS')

# for each day creates a list that counts and saves times of different events.
events_day_list_1 = []
events_day_list_shuffle_1 = []

events_counter_day_list = []
events_time_starts_day = []

for day in range(len(navigation_list)):
    events_list = []
    events_counter_list = []
    events_time_starts = []
    random_events = []
    start_counter = 100
    counter = 0
    #for i in range(exploration_list[day].shape[0]):
    for i in range(navigation_list[day].shape[0]):
        #if exploration_list[day][i] != start_counter:
        if navigation_list[day][i] != start_counter:
            events_list.append(start_counter) # contains a sequence of events ID
            events_counter_list.append(counter) # conteins the duration of each event
            events_time_starts.append(i) # contains the time at which event starts
            #start_counter = behaviour_list[day][i] 
            #start_counter = exploration_list[day][i]
            start_counter = navigation_list[day][i]
            counter = 1
        else:
            counter = counter + 1    
            
            
    events_day_list_1.append(events_list)
    shufflings = []
    for j in range(N_SHUFFLINGS):
        shufflings.append(events_list.copy())
    events_day_list_shuffle_1.append(shufflings)
    events_counter_day_list.append(events_counter_list)
    events_time_starts_day.append(events_time_starts)

CREATES LIST THAT SAVES ALL THE EVENTS IN A DAY AND CONTAINS ONSET OF VISITS


In [26]:
print('NOW WE SEPARATE EVENTS TYPES ACCORDING TO CORNER/OBJECT VISIT')
## compute events duration for type of event. Here we create lists that we can access to know the duration of each ID event
## we will use this to balance the dataset
selection_target = [1,2,3,4]    # here we balcance the data only for this targets of behaviour
id_target = selection_target

events_duration_list = []
total_duration_list = []
number_of_events_list = []
events_id = []

for day in range(len(behaviour_list)):
    events_duration_day = []
    total_duration_day = []
    number_of_events_day = []
    events_id_day = []
    
    events = np.array(events_day_list_1[day])  # all events in a day (THIS IS EVENT ID)
    events_counter = np.array(events_counter_day_list[day]) #duration of all day events
    events_time = np.array(events_time_starts_day[day]) # start time of events in day

    for target in id_target:
        position_events = np.where(events == target)[0] # select events related to one specific ID
        events_duration_target = events_counter[position_events]   # take the duration of the events for that ID
        
        if(len(events_duration_target)):
            events_duration_day.append(events_duration_target)
            total_duration_day.append(np.sum(events_duration_target))
            number_of_events_day.append(len(events_duration_target ))
            events_id_day.append(target)
            
    events_duration_list.append(events_duration_day)
    total_duration_list.append(total_duration_day)
    number_of_events_list.append(number_of_events_day)
    events_id.append(events_id_day)
    

NOW WE SEPARATE EVENTS TYPES ACCORDING TO CORNER/OBJECT VISIT


In [27]:
print('BALANCING TO THE LOWER NUMBER OF VISITS')
### Balancing the number of events for selected targets 
events_day_list= []         # create a list with the selected index of the ID-list to make a balanced selection
events_number_list = []
events_day_list_shuffle = []
for day in range(len(behaviour_list)):
    
    arg_min_target_time = np.argmin(number_of_events_list[day])
    n_events = number_of_events_list[day][arg_min_target_time]
    events_number_list.append(n_events)
    events_list = []
    events_list_copy = []
    print('Number of events per day after balancing: ',n_events)
    #print(n_events)
    for target in range(len(events_id[day])):
        sorted_events = np.sort(events_duration_list[day][target]) #sort of events
        arg_sorted_events = np.argsort(events_duration_list[day][target]) #take the index sorted by duration of events
        selected_events = arg_sorted_events[0:n_events]   # take only the first (sorter duration) events
        events_list.append(selected_events)                           # save the position of long and balanced events
        events_list_copy.append(selected_events.copy())               # make a copy of this events to create a shuffle list
        
    events_day_list.append(events_list)                              #this list contains index that are selected from the list of index of a specific target
    events_day_list_shuffle.append(events_list_copy)
    

BALANCING TO THE LOWER NUMBER OF VISITS
Number of events per day after balancing:  11
Number of events per day after balancing:  34
Number of events per day after balancing:  25
Number of events per day after balancing:  25


In [28]:
print('CREATE SHUFFLE LABELS THAT PRESERVE BALANCE')

### create shuffled behavioural labels that preserve balance and temporal structure
### the shuffling will be in visits or events (so we shuffle the labels of the complete event)

for day in range(len(behaviour_list)):
    events_shuffle = []
    events = np.array(events_day_list_1[day])
    events_counter = np.array(events_counter_day_list[day])
    events_time = np.array(events_time_starts_day[day])
    arg_min_target_time = np.argmin(number_of_events_list[day])
    n_events = number_of_events_list[day][arg_min_target_time]
    
    for target in range(len(events_id[day])):
        #print(target)
        #print(len(events_day_list[day]))
        #print(events_day_list[day][target])
        all_events = np.where(events == events_id[day][target])[0]
        #print(len(all_events))
        position_events = all_events[events_day_list[day][target]] # select the balanced data  
        events_shuffle.append(position_events)
    
    for j in range(N_SHUFFLINGS):
        counter_permutations = 0
        for i in range(n_events):
            permutation = np.random.permutation(len(events_id[day]))
            #print(permutation)
            counter_permutations +=1
            for index in range(len(events_id[day])):
                events_day_list_shuffle_1[day][j][events_shuffle[index][i]] = events_id[day][permutation[index]]

CREATE SHUFFLE LABELS THAT PRESERVE BALANCE


In [29]:
print('TAKING NEURAL OR TRANSFORMED ACTIVITY FOR EACH EVENT ... and create list with that')

## put all events together and take neural activity from each event
events_activity_pre_norm= []
events_duration_list = []
events_activity_pca = []
events_activity_cca = []
events_activity_cca2 = []
events_activity_cca0 = []
events_activity_lda = []


for day in range(len(behaviour_list)):
    target_activity = []
    target_activity_pca = []
    target_activity_cca = []
    target_activity_cca2 = []
    target_activity_cca0 = []
    target_activity_lda = []
    
    events_duration_day = []
    
    events = np.array(events_day_list_1[day])
    events_counter = np.array(events_counter_day_list[day])
    events_time = np.array(events_time_starts_day[day])

    for target in range(len(events_id[day])):
        all_events = np.where(events == events_id[day][target])[0]
        #print(all_events)
        position_events = all_events[events_day_list[day][target]] ### this contains the balanced events
        
        events_duration = events_counter[position_events]   # convert to seconds
        time = events_time[position_events]
        i = 0
        event_target = []
        event_target_pca = []
        event_target_cca = []
        event_target_cca2 = []
        event_target_cca0 = []
        event_target_lda = []

        #events_duration_target = np.zeros(len(events_duration),)
        events_duration_target = []
        for event in events_duration:
            if event and time[i]-period >0 and time[i]+period < activity_list[day].shape[1]:
                local_activity = activity_list[day][:,time[i]-period:time[i]+period]
                local_activity_pca = activity_list_pca[day][:,time[i]-period:time[i]+period]
                local_activity_cca = activity_list_cca[day][time[i]-period:time[i]+period,:].T
                local_activity_cca2 = activity_list_cca2[day][time[i]-period:time[i]+period,:].T
                local_activity_cca0 = activity_list_cca0[day][time[i]-period:time[i]+period,:].T
                local_activity_lda = activity_list_lda[day][:,time[i]-period:time[i]+period]

                event_target.append(local_activity)
                event_target_pca.append(local_activity_pca)
                event_target_cca.append(local_activity_cca)
                event_target_cca2.append(local_activity_cca2)
                event_target_cca2.append(local_activity_cca0)               
                event_target_lda.append(local_activity_lda)
                #events_duration_target[i]=1
                events_duration_target.append(events_duration[i])
            i = i + 1
        target_activity.append(event_target)
        target_activity_pca.append(event_target_pca)
        target_activity_cca.append(event_target_cca)
        target_activity_cca2.append(event_target_cca2)
        target_activity_cca0.append(event_target_cca0)
        target_activity_lda.append(event_target_lda)
        events_duration_day.append(events_duration_target)
        
    events_activity_pre_norm.append(target_activity)
    events_duration_list.append(events_duration_day)
    events_activity_pca.append(target_activity_pca)
    events_activity_cca.append(target_activity_cca)    
    events_activity_cca2.append(target_activity_cca2)
    events_activity_cca0.append(target_activity_cca0)    
    events_activity_lda.append(target_activity_lda)


TAKING NEURAL OR TRANSFORMED ACTIVITY FOR EACH EVENT ... and create list with that


In [132]:

print('TAKING NEURAL OR TRANSFORMED ACTIVITY FOR EACH EVENT ... and create list with that IN THE SHUFFLING DATA')

### same but with the events shuffle data

events_activity_pre_norm_shuffle= []
events_activity_pca_shuffle = []
events_activity_cca_shuffle = []
events_activity_cca_shuffle2 = []
events_activity_cca_shuffle0 = []
events_activity_lda_shuffle = []

for day in range(len(behaviour_list)):
    
    shufflings = []
    shufflings_pca = []
    shufflings_cca = []
    shufflings_cca2 = []
    shufflings_cca0 = []
    shufflings_lda = []

    for j in range(N_SHUFFLINGS):
        target_activity_shuffle = []
        target_activity_pca_shuffle = []
        target_activity_cca_shuffle = []
        target_activity_cca2_shuffle = []
        target_activity_cca0_shuffle = []
        target_activity_lda_shuffle = []
        
        events_duration_day_shuffle = []
        for target in range(len(events_id[day])):
            events = np.array(events_day_list_shuffle_1[day][j])
            events_counter = np.array(events_counter_day_list[day])
            events_time = np.array(events_time_starts_day[day])

            all_events = np.where(events == events_id[day][target])[0]

            #print(all_events)
            #print(events_day_list[day][target])
            position_events = all_events[events_day_list_shuffle[day][target]]

            events_duration = events_counter[position_events]   # convert to seconds
            time = events_time[position_events]
            i = 0
            event_target = []
            event_target_pca = []
            event_target_cca = []
            event_target_cca2 = []
            event_target_cca0 = []
            event_target_lda = []

            #events_duration_target = np.zeros(len(events_duration),)
            events_duration_target = []
            for event in events_duration:
                if event and time[i]-period >0 and time[i]+period < activity_list[day].shape[1]:
                    local_activity = activity_list[day][:,time[i]-period:time[i]+period]
                    local_activity_pca = activity_list_pca[day][:,time[i]-period:time[i]+period]
                    local_activity_cca = activity_list_cca[day][time[i]-period:time[i]+period,:].T
                    local_activity_cca2 = activity_list_cca2[day][time[i]-period:time[i]+period,:].T
                    local_activity_cca0 = activity_list_cca0[day][time[i]-period:time[i]+period,:].T
                    local_activity_lda = activity_list_lda[day][:,time[i]-period:time[i]+period]

                    event_target.append(local_activity)
                    event_target_pca.append(local_activity_pca)
                    event_target_cca.append(local_activity_cca)
                    event_target_cca2.append(local_activity_cca2)
                    event_target_cca0.append(local_activity_cca0)
                    event_target_lda.append(local_activity_lda)

                    #events_duration_target[i]=1
                    events_duration_target.append(events_duration[i])
                i = i + 1
            target_activity_shuffle.append(event_target)
            target_activity_pca_shuffle.append(event_target_pca)
            target_activity_cca_shuffle.append(event_target_cca)
            target_activity_cca2_shuffle.append(event_target_cca2)
            target_activity_cca0_shuffle.append(event_target_cca0)
            target_activity_lda_shuffle.append(event_target_lda)

        shufflings.append(target_activity_shuffle)
        shufflings_pca.append(target_activity_pca_shuffle)
        shufflings_cca.append(target_activity_cca_shuffle)
        shufflings_cca2.append(target_activity_cca2_shuffle)
        shufflings_cca0.append(target_activity_cca0_shuffle)
        shufflings_lda.append(target_activity_lda_shuffle)

    events_activity_pre_norm_shuffle.append(shufflings)
    events_activity_pca_shuffle.append(shufflings_pca)
    events_activity_cca_shuffle.append(shufflings_cca)
    events_activity_cca_shuffle2.append(shufflings_cca2)
    events_activity_cca_shuffle0.append(shufflings_cca0)
    events_activity_lda_shuffle.append(shufflings_lda)

events_activity = events_activity_pre_norm

TAKING NEURAL OR TRANSFORMED ACTIVITY FOR EACH EVENT ... and create list with that IN THE SHUFFLING DATA


In [133]:
lda_components = 4
for day in range(len(behaviour_list)): 

    print('Computing Trial matrices in all spaces for DAY = ' + str(day) )
    trial_activity_vectors = np.zeros((len(events_id[day]),events_activity[day][0][0].shape[0],events_activity[day][0][0].shape[1]))
    trial_activity_pca_vectors = np.zeros((len(events_id[day]),events_activity[day][0][0].shape[0],events_activity[day][0][0].shape[1]))
    trial_activity_cca_vectors = np.zeros((len(events_id[day]),cca_components,events_activity[day][0][0].shape[1])) 
    trial_activity_cca_vectors2 = np.zeros((len(events_id[day]),cca_components2,events_activity[day][0][0].shape[1])) 
    trial_activity_lda_vectors = np.zeros((len(events_id[day]),lda_components,events_activity[day][0][0].shape[1])) 

    j= 0    
    for target in range(len(events_id[day])):
        # real data! 
        print('Target:' + str(target))
        print('NUMBER OF VISITS:' + str(len(events_activity[day][target])))

        if events_activity[day][target] != []:

            trial_activity = np.zeros((events_activity[day][target][0].shape[0],events_activity[day][target][0].shape[1]))
            trial_activity_pca = np.zeros((events_activity[day][target][0].shape[0],events_activity[day][target][0].shape[1]))
            trial_activity_cca = np.zeros((cca_components,events_activity[day][target][0].shape[1]))
            trial_activity_cca2 = np.zeros((cca_components2,events_activity[day][target][0].shape[1]))
            trial_activity_lda = np.zeros((lda_components,events_activity[day][target][0].shape[1]))

            ### generate matrix with mean activity and entire trial repetitions activity
            for neuron in range(events_activity[day][target][0].shape[0]):
                neuron_trial_activity = np.zeros((events_activity[day][target][1].shape[1],))
                component_trial_activity = np.zeros((events_activity[day][target][0].shape[1],))
                cca_trial_activity = np.zeros((events_activity[day][target][0].shape[1],))
                cca_trial_activity2 = np.zeros((events_activity[day][target][0].shape[1],))
                lda_trial_activity = np.zeros((events_activity[day][target][0].shape[1],))
                for trial in range(len(events_activity[day][target])):
                    if len(events_activity[day][target][trial][neuron,:]):
                        neuron_trial_activity += events_activity[day][target][trial][neuron,:]#/(np.max(events_activity[day][target][trial][neuron,:])-np.min(events_activity[day][target][trial][neuron,:]))
                        component_trial_activity +=events_activity_pca[day][target][trial][neuron,:]
                        if neuron < cca_components:
                            cca_trial_activity +=events_activity_cca[day][target][trial][neuron,:]
                        if neuron < cca_components2:
                            cca_trial_activity2 +=events_activity_cca2[day][target][trial][neuron,:]
                        if neuron < lda_components:
                            lda_trial_activity +=events_activity_lda[day][target][trial][neuron,:]      
                neuron_trial_activity = neuron_trial_activity / len(events_activity[day][target])
                component_trial_activity = component_trial_activity / len(events_activity[day][target])
                trial_activity[neuron,:] = neuron_trial_activity
                trial_activity_pca[neuron,:] = component_trial_activity
                if neuron < cca_components:
                    trial_activity_cca[neuron,:] = cca_trial_activity / cca_components
                if neuron < cca_components2:
                    trial_activity_cca2[neuron,:] = cca_trial_activity2 / cca_components2
                if neuron < lda_components:
                    trial_activity_lda[neuron,:] = lda_trial_activity / lda_components

        trial_activity_vectors[j,:,:] = trial_activity
        trial_activity_pca_vectors[j,:,:] = trial_activity_pca
        trial_activity_cca_vectors[j,:,:] = trial_activity_cca[:cca_components,:]
        trial_activity_cca_vectors2[j,:,:] = trial_activity_cca2[:cca_components2,:]
        trial_activity_lda_vectors[j,:,:] = trial_activity_lda[:lda_components,:]


        j= j+1            
    print('Computing Trial matrices in all spaces for SHUFFLINGS')                
    trial_activity_vectors_shuffle = np.zeros((N_SHUFFLINGS,len(events_id[day]),events_activity[day][0][0].shape[0],events_activity[day][0][0].shape[1]))
    trial_activity_pca_vectors_shuffle = np.zeros((N_SHUFFLINGS,len(events_id[day]),events_activity[day][0][0].shape[0],events_activity[day][0][0].shape[1]))
    trial_activity_cca_vectors_shuffle = np.zeros((N_SHUFFLINGS,len(events_id[day]),cca_components,events_activity[day][0][0].shape[1]))
    trial_activity_cca_vectors_shuffle2 = np.zeros((N_SHUFFLINGS,len(events_id[day]),cca_components2,events_activity[day][0][0].shape[1]))
    trial_activity_lda_vectors_shuffle = np.zeros((N_SHUFFLINGS,len(events_id[day]),lda_components,events_activity[day][0][0].shape[1]))

    for shuffle in range(N_SHUFFLINGS):
        j=0
        for target in range(len(events_id[day])):
            trial_activity = np.zeros((events_activity_pre_norm_shuffle[day][shuffle][target][0].shape[0],events_activity_pre_norm_shuffle[day][shuffle ][target][0].shape[1]))
            trial_activity_pca = np.zeros((events_activity_pre_norm_shuffle[day][shuffle][target][0].shape[0],events_activity_pre_norm_shuffle[day][shuffle][target][0].shape[1]))
            trial_activity_cca = np.zeros((cca_components,events_activity_pre_norm_shuffle[day][shuffle][target][0].shape[1]))
            trial_activity_cca2 = np.zeros((cca_components2,events_activity_pre_norm_shuffle[day][shuffle][target][0].shape[1]))
            trial_activity_lda = np.zeros((lda_components,events_activity_pre_norm_shuffle[day][shuffle][target][0].shape[1]))

            ### generate matrix with mean activity and entire trial repetitions activity
            for neuron in range(events_activity_pre_norm_shuffle[day][shuffle][target][0].shape[0]):
                neuron_trial_activity = np.zeros((events_activity_pre_norm_shuffle[day][shuffle][target][0].shape[1],))
                component_trial_activity = np.zeros((events_activity_pre_norm_shuffle[day][shuffle][target][0].shape[1],))
                cca_trial_activity = np.zeros((events_activity_pre_norm_shuffle[day][shuffle][target][0].shape[1],))
                cca_trial_activity2 = np.zeros((events_activity_pre_norm_shuffle[day][shuffle][target][0].shape[1],))
                lda_trial_activity = np.zeros((events_activity_pre_norm_shuffle[day][shuffle][target][0].shape[1],))
                for trial in range(len(events_activity_pre_norm_shuffle[day][shuffle][target])):
                    if len(events_activity_pre_norm_shuffle[day][shuffle][target][trial][neuron,:]):
                        neuron_trial_activity += events_activity_pre_norm_shuffle[day][shuffle][target][trial][neuron,:]#/(np.max(events_activity[day][target][trial][neuron,:])-np.min(events_activity[day][target][trial][neuron,:]))
                        component_trial_activity +=events_activity_pca_shuffle[day][shuffle][target][trial][neuron,:]
                        if neuron < cca_components:
                            cca_trial_activity +=events_activity_cca_shuffle[day][shuffle][target][trial][neuron,:]
                        if neuron < cca_components2:
                            cca_trial_activity2 +=events_activity_cca_shuffle2[day][shuffle][target][trial][neuron,:]
                        if neuron < lda_components:
                            lda_trial_activity +=events_activity_lda_shuffle[day][shuffle][target][trial][neuron,:]

                neuron_trial_activity = neuron_trial_activity / len(events_activity_pre_norm_shuffle[day][shuffle][target])
                component_trial_activity = component_trial_activity / len(events_activity_pre_norm_shuffle[day][shuffle][target])
                trial_activity[neuron,:] = neuron_trial_activity
                trial_activity_pca[neuron,:] = component_trial_activity
                if neuron < cca_components:
                    trial_activity_cca[neuron,:] = cca_trial_activity / cca_components
                if neuron < cca_components2:
                    trial_activity_cca2[neuron,:] = cca_trial_activity2 / cca_components
                if neuron < lda_components:
                    trial_activity_lda[neuron,:] = lda_trial_activity / lda_components

            trial_activity_vectors_shuffle[shuffle,j,:,:] = trial_activity
            trial_activity_pca_vectors_shuffle[shuffle,j,:,:] = trial_activity_pca
            trial_activity_cca_vectors_shuffle[shuffle,j,:,:] = trial_activity_cca[:cca_components,:]
            trial_activity_cca_vectors_shuffle2[shuffle,j,:,:] = trial_activity_cca2[:cca_components2,:]
            trial_activity_lda_vectors_shuffle[shuffle,j,:,:] = trial_activity_lda[:lda_components,:]
            j=j+1


    print('Computing representational distance in real data')
    distance_neural = np.zeros((trial_activity_pca_vectors.shape[2],))
    distance_pca = np.zeros((trial_activity_pca_vectors.shape[2],))
    distance_cca = np.zeros((trial_activity_cca_vectors.shape[2],))
    distance_cca2 = np.zeros((trial_activity_cca_vectors2.shape[2],))
    distance_lda = np.zeros((trial_activity_cca_vectors.shape[2],))

    pca_components = np.where(variance_ratio_list[day]>0.6)[0][0]

    for time in range(trial_activity_vectors.shape[2]):
        counter = 0
        for i in range(trial_activity_pca_vectors.shape[0]):
            for j in range(i+1,trial_activity_pca_vectors.shape[0]):
                distance_neural[time] += np.linalg.norm(trial_activity_vectors[i,:,time] - trial_activity_vectors[j,:,time])
                distance_pca[time] += np.linalg.norm(trial_activity_pca_vectors[i,0:pca_components,time] - trial_activity_pca_vectors[j,0:pca_components,time])
                distance_cca[time] += np.linalg.norm(trial_activity_cca_vectors[i,:,time] - trial_activity_cca_vectors[j,:,time])
                distance_cca2[time] += np.linalg.norm(trial_activity_cca_vectors2[i,:,time] - trial_activity_cca_vectors2[j,:,time])
                distance_lda[time] += np.linalg.norm(trial_activity_lda_vectors[i,:,time] - trial_activity_lda_vectors[j,:,time])

                counter+=1

        distance_neural[time] = distance_neural[time]/counter
        distance_pca[time] = distance_pca[time]/counter
        distance_cca[time] = distance_cca[time]/counter
        distance_cca2[time] = distance_cca2[time]/counter
        distance_lda[time] = distance_lda[time]/counter

    print('Computing representational distance in shuffling data')

    distance_neural_shuffle = np.zeros((N_SHUFFLINGS,trial_activity_vectors.shape[2],))
    distance_pca_shuffle = np.zeros((N_SHUFFLINGS,trial_activity_vectors.shape[2],))
    distance_cca_shuffle = np.zeros((N_SHUFFLINGS,trial_activity_vectors.shape[2],))
    distance_cca_shuffle2 = np.zeros((N_SHUFFLINGS,trial_activity_vectors.shape[2],))
    distance_lda_shuffle = np.zeros((N_SHUFFLINGS,trial_activity_vectors.shape[2],))

    for shuffle in range(N_SHUFFLINGS):
        for time in range(trial_activity_vectors.shape[2]):
            counter = 0
            for i in range(trial_activity_pca_vectors_shuffle.shape[1]):
                for j in range(i+1,trial_activity_pca_vectors_shuffle.shape[1]):
                    distance_neural_shuffle[shuffle,time] += np.linalg.norm(trial_activity_vectors_shuffle[shuffle,i,:,time] - trial_activity_vectors_shuffle[shuffle,j,:,time])
                    distance_pca_shuffle[shuffle,time] += np.linalg.norm(trial_activity_pca_vectors_shuffle[shuffle,i,0:pca_components,time] - trial_activity_pca_vectors_shuffle[shuffle,j,0:pca_components,time])
                    distance_cca_shuffle[shuffle,time] += np.linalg.norm(trial_activity_cca_vectors_shuffle[shuffle,i,:,time] - trial_activity_cca_vectors_shuffle[shuffle,j,:,time])
                    distance_cca_shuffle2[shuffle,time] += np.linalg.norm(trial_activity_cca_vectors_shuffle2[shuffle,i,:,time] - trial_activity_cca_vectors_shuffle2[shuffle,j,:,time])
                    distance_lda_shuffle[shuffle,time] += np.linalg.norm(trial_activity_lda_vectors_shuffle[shuffle,i,:,time] - trial_activity_lda_vectors_shuffle[shuffle,j,:,time])
                    counter = counter +1
            distance_neural_shuffle[shuffle,time] = distance_neural_shuffle[shuffle,time]/counter
            distance_pca_shuffle[shuffle,time] = distance_pca_shuffle[shuffle,time]/counter
            distance_cca_shuffle[shuffle,time] = distance_cca_shuffle[shuffle,time]/counter
            distance_cca_shuffle2[shuffle,time] = distance_cca_shuffle2[shuffle,time]/counter
            distance_lda_shuffle[shuffle,time] = distance_lda_shuffle[shuffle,time]/counter

    distance_mean = np.mean(distance_neural_shuffle,axis=0)
    distance_pca_mean = np.mean(distance_pca_shuffle,axis=0)
    distance_cca_mean = np.mean(distance_cca_shuffle,axis=0)
    distance_cca_mean2 = np.mean(distance_cca_shuffle2,axis=0)
    distance_lda_mean = np.mean(distance_lda_shuffle,axis=0)

    distance_std = np.std(distance_neural_shuffle,axis=0)
    distance_pca_std = np.std(distance_pca_shuffle,axis=0)
    distance_cca_std = np.std(distance_cca_shuffle,axis=0) 
    distance_cca_std2 = np.std(distance_cca_shuffle2,axis=0) 
    distance_lda_std = np.std(distance_lda_shuffle,axis=0)

    print('Z-SCORING distances')
    z_scored_neural = (distance_neural - distance_mean)/distance_std
    z_scored_pca = (distance_pca - distance_pca_mean)/distance_pca_std
    z_scored_cca = (distance_cca - distance_cca_mean)/distance_cca_std
    z_scored_cca2 = (distance_cca2 - distance_cca_mean2)/distance_cca_std
    z_scored_lda = (distance_lda - distance_lda_mean)/distance_lda_std

    final_distance = np.zeros((5, len(z_scored_neural)))
    final_distance[0,:] = z_scored_neural
    final_distance[1,:] = z_scored_pca
    final_distance[2,:] = z_scored_cca
    final_distance[3,:] = z_scored_cca2
    final_distance[4,:] = z_scored_lda

    print('Saving...')
    output_path =  os.environ['PROJECT_DIR'] +'neural_analysis/data/mean_representational_distance/'
    file_name = output_path  +'corners_position_distance_2_mouse_'+f'{mouse}'+'_session_'+f'{session}'+ '_day_'+f'{day}'
    np.save(file_name, final_distance)

print('DONE')

Computing Trial matrices in all spaces for DAY = 0
Target:0
NUMBER OF VISITS:36
Target:1
NUMBER OF VISITS:36
Target:2
NUMBER OF VISITS:36
Target:3
NUMBER OF VISITS:36
Computing Trial matrices in all spaces for SHUFFLINGS
Computing representational distance in real data
Computing representational distance in shuffling data
Z-SCORING distances
Saving...
Computing Trial matrices in all spaces for DAY = 1
Target:0
NUMBER OF VISITS:9
Target:1
NUMBER OF VISITS:9
Target:2
NUMBER OF VISITS:9
Target:3
NUMBER OF VISITS:9
Computing Trial matrices in all spaces for SHUFFLINGS
Computing representational distance in real data
Computing representational distance in shuffling data
Z-SCORING distances
Saving...
Computing Trial matrices in all spaces for DAY = 2
Target:0
NUMBER OF VISITS:10
Target:1
NUMBER OF VISITS:10
Target:2
NUMBER OF VISITS:10
Target:3
NUMBER OF VISITS:10
Computing Trial matrices in all spaces for SHUFFLINGS
Computing representational distance in real data
Computing representational